In [1]:
import pandas as pd
import numpy as np
import random

In [2]:
path = "C:/Users/chial/Downloads/data.xlsx" 
ads_table = pd.read_excel(path, sheet_name = 0, header = 1)
mod_table = pd.read_excel(path, sheet_name = 1)

In [3]:
ads_table_dropped = ads_table.dropna()
ads_table_dropped = ads_table_dropped.reset_index(drop=True)
ads_table_dropped

,p_date,ad_id,delivery_country,queue_market,punish_num,latest_punish_begin_date,ad_revenue,avg_ad_revenue,start_time,baseline_st,product_line,task_type_en
0,20230807,1747578422390810,US,USCA,1.0,2023-07-24,5132.1700,4795.2500,2022-10-24 14:36:00,1.78,Auction Ads,auction_relation
1,20230807,1758543228094480,SA,MENA,1.0,2023-06-14,1976.2500,1063.7670,2023-02-22 15:50:00,1.67,RIE,Ad Group Recall (Merged)
2,20230807,1738303151826990,AU,AU,2.0,2023-07-24,1.7355,5.3650,2022-07-14 05:00:00,1.84,Auction Ads,auction_relation
3,20230807,1738304016410620,NZ,NZ,2.0,2023-07-24,1.5250,5.3650,2022-07-14 05:17:00,1.58,Auction Ads,auction_relation
4,20230807,1738304421208090,NZ,NZ,2.0,2023-07-24,0.7889,5.3650,2022-07-14 05:24:00,1.58,Auction Ads,auction_relation
...,...,...,...,...,...,...,...,...,...,...,...,...
1280,20230807,1773610467447840,NO,Others,1.0,2023-06-26,0.3700,4.5148,2023-08-07 22:20:00,1.56,Auction Ads,auction_all
1281,20230807,1773560585140270,KH,KH,1.0,2023-05-16,0.1452,0.4317,2023-08-07 22:00:00,1.50,Auction Ads,auction_relation
1282,20230807,1773559790111770,KH,KH,1.0,2023-05-16,0.1352,0.4317,2023-08-07 22:00:00,1.50,Auction Ads,auction_relation
1283,20230807,1773595125935120,BR,BR,1.0,2023-07-17,0.0000,26.7034,2023-08-08 03:00:00,1.43,Non-Auction Ads,R&F


In [4]:
def fake_score():
    return random.random()

ads_table_dropped['score'] = 0
ads_table_dropped['score'] = ads_table_dropped['score'].apply(lambda x: fake_score())
ads_table_dropped

,p_date,ad_id,delivery_country,queue_market,punish_num,latest_punish_begin_date,ad_revenue,avg_ad_revenue,start_time,baseline_st,product_line,task_type_en,score
0,20230807,1747578422390810,US,USCA,1.0,2023-07-24,5132.1700,4795.2500,2022-10-24 14:36:00,1.78,Auction Ads,auction_relation,0.882213
1,20230807,1758543228094480,SA,MENA,1.0,2023-06-14,1976.2500,1063.7670,2023-02-22 15:50:00,1.67,RIE,Ad Group Recall (Merged),0.939488
2,20230807,1738303151826990,AU,AU,2.0,2023-07-24,1.7355,5.3650,2022-07-14 05:00:00,1.84,Auction Ads,auction_relation,0.113882
3,20230807,1738304016410620,NZ,NZ,2.0,2023-07-24,1.5250,5.3650,2022-07-14 05:17:00,1.58,Auction Ads,auction_relation,0.106942
4,20230807,1738304421208090,NZ,NZ,2.0,2023-07-24,0.7889,5.3650,2022-07-14 05:24:00,1.58,Auction Ads,auction_relation,0.827104
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1280,20230807,1773610467447840,NO,Others,1.0,2023-06-26,0.3700,4.5148,2023-08-07 22:20:00,1.56,Auction Ads,auction_all,0.839284
1281,20230807,1773560585140270,KH,KH,1.0,2023-05-16,0.1452,0.4317,2023-08-07 22:00:00,1.50,Auction Ads,auction_relation,0.978827
1282,20230807,1773559790111770,KH,KH,1.0,2023-05-16,0.1352,0.4317,2023-08-07 22:00:00,1.50,Auction Ads,auction_relation,0.356420
1283,20230807,1773595125935120,BR,BR,1.0,2023-07-17,0.0000,26.7034,2023-08-08 03:00:00,1.43,Non-Auction Ads,R&F,0.364650


## Queuing

In [5]:
mod_table['moderator'].nunique()==len(mod_table) # Check for duplicates in moderator

True

In [6]:
# Pass the review in as a job (dataframe of 1 row)
#Naive:
job = ads_table_dropped.iloc[0]
job

p_date                                 20230807
ad_id                          1747578422390810
delivery_country                             US
queue_market                               USCA
punish_num                                  1.0
latest_punish_begin_date    2023-07-24 00:00:00
ad_revenue                              5132.17
avg_ad_revenue                          4795.25
start_time                  2022-10-24 14:36:00
baseline_st                                1.78
product_line                        Auction Ads
task_type_en                   auction_relation
score                                  0.882213
Name: 0, dtype: object

In [7]:
# Check moderator's market
def check_moderator_market(target_market):
    return mod_table[mod_table['market'].apply(lambda x: target_market in x)]

moderator_q = check_moderator_market(job['delivery_country'])
moderator_q

,moderator,market,Productivity,Utilisation %,handling time,accuracy
54,5608898,"[""US"", ""CA""]",565.187500,0.811286,106304,0.963
56,1672502123346950,"[""US"", ""CA""]",133.226667,0.537611,79835,0.789
59,1752289495219201,"[""US"", ""CA""]",383.448000,0.805644,97686,0.911
71,1759961674413106,"[""US""]",261.977500,0.808214,89468,0.792
78,2873783,"[""US"", ""CA""]",204.236667,0.529403,89440,0.455
...,...,...,...,...,...,...
1378,1598527,"[""US"", ""CA""]",1.230000,0.000000,73,0.676
1388,9939125,"[""US"", ""CA""]",317.657143,0.384768,37685,0.38
1394,1251392,"[""US""]",615.867500,0.734984,95137,0.438
1406,7889705,"[""US"", ""CA""]",336.700000,0.890469,112190,0.615


In [8]:
mod_table

,moderator,market,Productivity,Utilisation %,handling time,accuracy
0,1689841547143170,"[""SA"", ""OM"", ""BH"", ""QA"", ""JO"", ""IQ"", ""KW"", ""EG...",286.217674,0.812403,123549,-
1,1686755036370945,"[""RO""]",174.560000,0.704833,25547,0.959
2,1741913197768705,"[""VN""]",334.376000,0.819167,50794,1
3,1743415203890193,"[""MX"", ""UY"", ""CL"", ""AR"", ""CO"", ""EC"", ""PE"", ""BR""]",452.277500,0.775474,89626,0.941
4,1710346282427393,"[""PH""]",612.312500,0.843229,98066,0.887
...,...,...,...,...,...,...
1409,1734911746733057,"[""US"", ""CA""]",180.750000,0.685469,79210,0.86
1410,1694737747315714,"[""TH""]",542.360000,0.890927,49664,0.958
1411,1730993346971650,"[""RO""]",320.960000,0.843042,73769,0.967
1412,1707706649725953,"[""GB""]",283.261333,0.846072,90975,0.865


In [ ]:
# I need a function where I input the list of moderators, and you return me their job queues

# Gist of how the queuing gonna work:
# 1) Split moderators into 3 segments: highaccuracy, midaccuracy,lowaccuracy
# 2) Split jobs into 3 segments: highrisk, midrisk, lowrisk
# 3) High risk jobs to be assigned to the respective segments to highaccuracy moderators, lowrisk jobs for lowaccuracy moderators etc
# 4) From the segments, check their job queue to determine which moderator to assign to 

# Arbitrarily, top 25% is high accuracy, next 50% is mid accuracy, bottom 25% is low accuracy
# Arbitrarily, 0.76 - 1.00 is high risk, 0.25 - 0.75 is mid risk, 0.00 - 0.24 is low risk    

In [9]:
# 1) Split moderators into 3 segments: highaccuracy, midaccuracy,lowaccuracy
high_accuracy_cutoff = moderator_q['accuracy'].quantile(0.75)
mid_accuracy_cutoff = moderator_q['accuracy'].quantile(0.25)

moderator_q['Accuracy Segment'] = np.where(moderator_q['accuracy'] >= high_accuracy_cutoff, 2,
                         np.where(moderator_q['accuracy'] >= mid_accuracy_cutoff, 1, 0))

moderator_q

C:\Users\chial\AppData\Local\Temp/ipykernel_16312/3536365137.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  moderator_q['Accuracy Segment'] = np.where(moderator_q['accuracy'] >= high_accuracy_cutoff, 2,


,moderator,market,Productivity,Utilisation %,handling time,accuracy,Accuracy Segment
54,5608898,"[""US"", ""CA""]",565.187500,0.811286,106304,0.963,2
56,1672502123346950,"[""US"", ""CA""]",133.226667,0.537611,79835,0.789,1
59,1752289495219201,"[""US"", ""CA""]",383.448000,0.805644,97686,0.911,1
71,1759961674413106,"[""US""]",261.977500,0.808214,89468,0.792,1
78,2873783,"[""US"", ""CA""]",204.236667,0.529403,89440,0.455,0
...,...,...,...,...,...,...,...
1378,1598527,"[""US"", ""CA""]",1.230000,0.000000,73,0.676,1
1388,9939125,"[""US"", ""CA""]",317.657143,0.384768,37685,0.38,0
1394,1251392,"[""US""]",615.867500,0.734984,95137,0.438,0
1406,7889705,"[""US"", ""CA""]",336.700000,0.890469,112190,0.615,0


In [10]:
moderator_q['Accuracy Segment'].value_counts()

1    80
2    41
0    40
Name: Accuracy Segment, dtype: int64

In [11]:
# 2) Split jobs into 3 segments: highrisk, midrisk, lowrisk
def tag_job_with_risk (job):
    if job['score']>0.75:
        return 2
    elif job['score']>0.25:
        return 1
    else: return 0

job['Risk Segment'] = tag_job_with_risk(job)
job

C:\Users\chial\AppData\Roaming\Python\Python39\site-packages\pandas\core\series.py:1056: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cacher_needs_updating = self._check_is_chained_assignment_possible()
C:\Users\chial\AppData\Roaming\Python\Python39\site-packages\pandas\core\indexing.py:1724: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer_missing(indexer, value)


p_date                                 20230807
ad_id                          1747578422390810
delivery_country                             US
queue_market                               USCA
punish_num                                  1.0
latest_punish_begin_date    2023-07-24 00:00:00
ad_revenue                              5132.17
avg_ad_revenue                          4795.25
start_time                  2022-10-24 14:36:00
baseline_st                                1.78
product_line                        Auction Ads
task_type_en                   auction_relation
score                                  0.882213
Risk Segment                                  2
Name: 0, dtype: object

In [12]:
moderator_q.drop(moderator_q[moderator_q['Accuracy Segment'] != job['Risk Segment']].index, inplace=True)
moderator_q

C:\Users\chial\AppData\Roaming\Python\Python39\site-packages\pandas\core\frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


,moderator,market,Productivity,Utilisation %,handling time,accuracy,Accuracy Segment
54,5608898,"[""US"", ""CA""]",565.187500,0.811286,106304,0.963,2
114,7752505,"[""US"", ""CA""]",656.727500,0.807500,95546,1,2
181,1717152819047426,"[""US"", ""CA""]",NaN,NaN,0,0.955,2
242,1669223,"[""US"", ""CA""]",293.406667,0.833333,69150,1,2
294,8895709,"[""US"", ""CA""]",270.541177,0.244696,96160,1,2
295,5081203,"[""US"", ""CA""]",NaN,NaN,0,1,2
318,8750521,"[""US"", ""CA""]",689.007500,0.860505,114756,0.968,2
372,9832298,"[""US"", ""CA""]",NaN,NaN,0,0.956,2
388,6085577,"[""US"", ""CA""]",626.010000,0.976161,110861,0.949,2
435,5311062,"[""US"", ""CA""]",554.687500,0.919844,104661,0.963,2


In [13]:
# Function to return number of jobs given ID of moderator
def func(id):
    return 1

moderator_q['Number of Jobs'] = 0
moderator_q['Number of Jobs'] = moderator_q['Number of Jobs'].apply(lambda x: func(moderator_q['moderator']))

C:\Users\chial\AppData\Local\Temp/ipykernel_16312/2242769195.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  moderator_q['Number of Jobs'] = 0
C:\Users\chial\AppData\Local\Temp/ipykernel_16312/2242769195.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  moderator_q['Number of Jobs'] = moderator_q['Number of Jobs'].apply(lambda x: func(moderator_q['moderator']))


In [16]:
## Least Job Policy
least_jobs_value = moderator_q['Number of Jobs'].min()
moderator = moderator_q.loc[moderator_q['Number of Jobs'] == least_jobs_value, 'moderator'].values[0]
moderator

5608898

In [15]:
## Least Job Hour Policy
moderator_q['Number of Job Hours'] = moderator_q['Number of Jobs']*moderator_q['handling time']
least_jobs_value = moderator_q['Number of Job Hours'].min()
moderator = moderator_q.loc[moderator_q['Number of Job Hours'] == least_jobs_value, 'moderator'].values[0]
moderator

C:\Users\chial\AppData\Local\Temp/ipykernel_16312/844172576.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  moderator_q['Number of Job Hours'] = moderator_q['Number of Jobs']*moderator_q['handling time']


1717152819047426

In [17]:
# Need to clean the moderator data. Productivity and Utilisation cant be NaN and hadling time cannot be 0 
moderator_q

,moderator,market,Productivity,Utilisation %,handling time,accuracy,Accuracy Segment,Number of Jobs,Number of Job Hours
54,5608898,"[""US"", ""CA""]",565.187500,0.811286,106304,0.963,2,1,106304
114,7752505,"[""US"", ""CA""]",656.727500,0.807500,95546,1,2,1,95546
181,1717152819047426,"[""US"", ""CA""]",NaN,NaN,0,0.955,2,1,0
242,1669223,"[""US"", ""CA""]",293.406667,0.833333,69150,1,2,1,69150
294,8895709,"[""US"", ""CA""]",270.541177,0.244696,96160,1,2,1,96160
295,5081203,"[""US"", ""CA""]",NaN,NaN,0,1,2,1,0
318,8750521,"[""US"", ""CA""]",689.007500,0.860505,114756,0.968,2,1,114756
372,9832298,"[""US"", ""CA""]",NaN,NaN,0,0.956,2,1,0
388,6085577,"[""US"", ""CA""]",626.010000,0.976161,110861,0.949,2,1,110861
435,5311062,"[""US"", ""CA""]",554.687500,0.919844,104661,0.963,2,1,104661
